# Лабороторная работа 1

## Задание 1

Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).

### Импорт библиотек

In [10]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


### Подключение гугл диска

In [11]:
%%time
# монтируем диск
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CPU times: user 60 ms, sys: 1.94 ms, total: 61.9 ms
Wall time: 3.5 s


In [12]:
df = pd.read_csv('/content/drive/MyDrive/cars.csv')
print(df.head(5))

  manufacturer_name model_name transmission   color  odometer_value  \
0            Subaru    Outback    automatic  silver          190000   
1            Subaru    Outback    automatic    blue          290000   
2            Subaru   Forester    automatic     red          402000   
3            Subaru    Impreza   mechanical    blue           10000   
4            Subaru     Legacy    automatic   black          280000   

   year_produced engine_fuel  engine_has_gas engine_type  engine_capacity  \
0           2010    gasoline           False    gasoline              2.5   
1           2002    gasoline           False    gasoline              3.0   
2           2001    gasoline           False    gasoline              2.5   
3           1999    gasoline           False    gasoline              3.0   
4           2001    gasoline           False    gasoline              2.5   

   ... feature_1  feature_2 feature_3 feature_4  feature_5  feature_6  \
0  ...      True       True      True

### Удаление ненужных и пустых столбцов

In [13]:
columns_to_drop = ['feature_0','feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'duration_listed']
df = df.drop(columns=columns_to_drop, axis=1)
df.dropna(inplace=True)

print(df.head(5))

  manufacturer_name model_name transmission   color  odometer_value  \
0            Subaru    Outback    automatic  silver          190000   
1            Subaru    Outback    automatic    blue          290000   
2            Subaru   Forester    automatic     red          402000   
3            Subaru    Impreza   mechanical    blue           10000   
4            Subaru     Legacy    automatic   black          280000   

   year_produced engine_fuel  engine_has_gas engine_type  engine_capacity  \
0           2010    gasoline           False    gasoline              2.5   
1           2002    gasoline           False    gasoline              3.0   
2           2001    gasoline           False    gasoline              2.5   
3           1999    gasoline           False    gasoline              3.0   
4           2001    gasoline           False    gasoline              2.5   

   body_type  has_warranty  state drivetrain  price_usd  is_exchangeable  \
0  universal         False  owned 

### Кодирование текстовых значений

In [14]:
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

print(df.head(5))

   manufacturer_name  model_name  transmission  color  odometer_value  \
0                 45         763             0      8          190000   
1                 45         763             0      1          290000   
2                 45         519             0      7          402000   
3                 45         609             1      1           10000   
4                 45         664             0      0          280000   

   year_produced  engine_fuel  engine_has_gas  engine_type  engine_capacity  \
0           2010            2           False            1              2.5   
1           2002            2           False            1              3.0   
2           2001            2           False            1              2.5   
3           1999            2           False            1              3.0   
4           2001            2           False            1              2.5   

   body_type  has_warranty  state  drivetrain  price_usd  is_exchangeable  \
0        

### Стандартизация датасета

In [15]:
scaler = StandardScaler()
scaler.fit(df.drop('is_exchangeable', axis=1))
scaled_features = scaler.transform(df.drop('is_exchangeable', axis=1))
scaled_data = pd.DataFrame(scaled_features, columns = df.drop('is_exchangeable', axis=1).columns)

print(scaled_data.head(5))

   manufacturer_name  model_name  transmission     color  odometer_value  \
0           1.089297    0.597314     -1.410285  0.978100       -0.432979   
1           1.089297    0.597314     -1.410285 -0.969685        0.302004   
2           1.089297   -0.149960     -1.410285  0.699845        1.125184   
3           1.089297    0.125674      0.709076 -0.969685       -1.755946   
4           1.089297    0.294117     -1.410285 -1.247940        0.228505   

   year_produced  engine_fuel  engine_has_gas  engine_type  engine_capacity  \
0       0.875318     0.721021       -0.190355     0.708498         0.662782   
1      -0.116665     0.721021       -0.190355     0.708498         1.407751   
2      -0.240663     0.721021       -0.190355     0.708498         0.662782   
3      -0.488659     0.721021       -0.190355     0.708498         1.407751   
4      -0.240663     0.721021       -0.190355     0.708498         0.662782   

   body_type  has_warranty     state  drivetrain  price_usd  locatio

### Разделение датасета на обучающие и тестовые данные

In [16]:
x = scaled_data
y = df['is_exchangeable']

x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(x, y, test_size = 0.3)

### Устанавливаем параметры для подбора


In [17]:
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

## Задание 2 .
Провести классификацию найденного датасета, методом к- ближайших соседей.

### Создаем модель и выполняем GridSearchCV

In [18]:
knn = KNeighborsClassifier()
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
start_time = time.time()
grid_search.fit(x_training_data, y_training_data)
end_time = time.time()

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [19]:
print(f"GridSearchCV took {end_time - start_time:.2f} seconds")
print("Best parameters found:", grid_search.best_params_)

GridSearchCV took 434.72 seconds
Best parameters found: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'distance'}


### Обучаем модель с лучшими параметрами

In [20]:
best_knn = grid_search.best_estimator_
predictions = best_knn.predict(x_test_data)

### Выводим метрики качества модели

In [21]:
print(classification_report(y_test_data, predictions))
print("Confusion Matrix:")
print(confusion_matrix(y_test_data, predictions))
print("Accuracy:", accuracy_score(y_test_data, predictions))

              precision    recall  f1-score   support

       False       0.68      0.90      0.77      7443
        True       0.56      0.23      0.33      4114

    accuracy                           0.66     11557
   macro avg       0.62      0.56      0.55     11557
weighted avg       0.64      0.66      0.61     11557

Confusion Matrix:
[[6696  747]
 [3170  944]]
Accuracy: 0.6610712122523146
